In [ ]:
!rm -rf Dataset Trained_Model                   # Remove existing directory
!pip install -U gdown --pre >/dev/null          # Install gdown to download file from GDrive
!gdown 1u4WJLjYrbZHwdvFOHQXJqDTtco6F5hJ-        # Download dataset from GDrive by file ID
!unzip -q Dataset.zip; rm Dataset.zip           # Extract the dataset zip file

In [ ]:
import os
import cv2
import numpy as np
from skimage.io import imread
from sklearn.model_selection import train_test_split


IMAGES_PATH = 'Dataset/Images/'
MASKS_PATH  = 'Dataset/Masks/'
TEST_PATH   = 'Dataset/Test_Images/'

# Number of images to use (Larger the number, more RAM required)
N_IMAGES = 1500

# Imread each image and save to an array

sat_imgs = os.listdir(IMAGES_PATH)
msk_imgs = os.listdir(MASKS_PATH)
sat_imgs.sort(), msk_imgs.sort()

images = []
for image in (sat_imgs[:N_IMAGES]):
    data = imread(IMAGES_PATH + image)
    images.append(data)

masks = []
for mask in (msk_imgs[:N_IMAGES]):
    data = imread(MASKS_PATH + mask)
    data = cv2.cvtColor(data, cv2.COLOR_BGR2GRAY)
    data = np.expand_dims(data, axis=-1)
    masks.append(data)

images = np.stack(images)
masks = np.stack(masks) / 255

train_images, test_images, train_masks, test_masks = train_test_split(images, masks, test_size=0.2, random_state=2)
del images, masks

print("Training Set")
print(train_images.shape)
print(train_masks.shape)
print("Testing Set")
print(test_images.shape)
print(test_masks.shape)

!rm -rf epochs Trained_Model; mkdir epochs

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf

resolver = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [ ]:
from tensorflow.keras import backend as K
def iou_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou

def dice_coef(y_true, y_pred, smooth = 1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def soft_dice_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [ ]:
# Import Libraries
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, Dropout, Activation, BatchNormalization
from tensorflow.keras.models import Model

def FCN32():

    img_input = Input(shape=(512, 512, 3))

    x = Conv2D(64, 3, activation='relu', name='Block-1_Conv-1', padding='same') (img_input)
    x = BatchNormalization() (x); x = Dropout(0.1) (x)
    x = Conv2D(64, 3, activation='relu', name='Block-1_Conv-2', padding='same') (x)
    x = BatchNormalization() (x); x = Dropout(0.1) (x)
    x = MaxPooling2D(2, strides=2, name='P-1') (x)

    # Block 2
    x = Conv2D(128, 3, activation='relu', name='Block-2_Conv-1', padding='same') (x)
    x = BatchNormalization() (x); x = Dropout(0.2) (x)
    x = Conv2D(128, 3, activation='relu', name='Block-2_Conv-2', padding='same') (x)
    x = BatchNormalization() (x); x = Dropout(0.2) (x)
    x = MaxPooling2D(2, strides=2, name='P-2') (x)

    # Block 3
    x = Conv2D(256, 3, activation='relu', name='Block-3_Conv-1', padding='same') (x)
    x = BatchNormalization() (x); x = Dropout(0.3) (x)
    x = Conv2D(256, 3, activation='relu', name='Block-3_Conv-2', padding='same') (x)
    x = BatchNormalization() (x); x = Dropout(0.3) (x)
    x = Conv2D(256, 3, activation='relu', name='Block-3_Conv-3', padding='same') (x)
    x = BatchNormalization() (x); x = Dropout(0.3) (x)
    x = MaxPooling2D(2, strides=2, name='P-3') (x)
    

    # Block 4
    x = Conv2D(512, 3, activation='relu', name='Block-4_Conv-1', padding='same') (x)
    x = BatchNormalization() (x); x = Dropout(0.3) (x)
    x = Conv2D(512, 3, activation='relu', name='Block-4_Conv-2', padding='same') (x)
    x = BatchNormalization() (x); x = Dropout(0.3) (x)
    x = Conv2D(512, 3, activation='relu', name='Block-4_Conv-3', padding='same') (x)
    x = BatchNormalization() (x); x = Dropout(0.3) (x)
    x = MaxPooling2D(2, strides=2, name='P-4') (x)
    

    # Block 5
    x = Conv2D(512, 3, activation='relu', name='Block-5_Conv-1', padding='same') (x)
    x = BatchNormalization() (x); x = Dropout(0.2) (x)
    x = Conv2D(512, 3, activation='relu', name='Block-5_Conv-2', padding='same') (x)
    x = BatchNormalization() (x); x = Dropout(0.2) (x)
    x = Conv2D(512, 3, activation='relu', name='Block-5_Conv-3', padding='same') (x)
    x = BatchNormalization() (x); x = Dropout(0.2) (x)
    x = MaxPooling2D(2, strides=2, name='P-5') (x)


    x = Conv2D(4096 , (7, 7) , activation='relu' , name='Fully-Connected-1', padding='same') (x)
    x = Conv2D(4096 , (1, 1) , activation='relu' , name='Fully-Connected-2', padding='same') (x)

    
    # Upsampling Layers

    x = Conv2DTranspose(128, kernel_size=2, kernel_initializer='he_normal', name='Upsample_2x', strides=2) (x)
    x = Conv2DTranspose( 64, kernel_size=2, kernel_initializer='he_normal', name='Upsample_4x', strides=2) (x)
    x = Conv2DTranspose(128, kernel_size=2, kernel_initializer='he_normal', name='Upsample_8x', strides=2) (x)
    x = Conv2DTranspose( 64, kernel_size=2, kernel_initializer='he_normal', name='Upsample_16x', strides=2) (x)
    x = Conv2DTranspose( 32, kernel_size=2, kernel_initializer='he_normal', name='Upsample_32x', strides=2) (x)

    x = Conv2D(1, 1, kernel_initializer='he_normal') (x)
    x = Dropout(0.1) (x)

    x = (Activation('sigmoid'))(x)
    model = Model(img_input, x)
    return model

In [ ]:
with strategy.scope():
    model = FCN32()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import backend as K
from IPython.display import clear_output
from skimage.io import imread, imshow
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import tensorflow as tf

model_path = "./Trained_Model/Road_Model.h5"


checkpointer = ModelCheckpoint(model_path, monitor="val_loss", mode="min", save_best_only = True, verbose=1)
earlystopper = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 10, verbose = 1, restore_best_weights = True)
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_delta=1e-4)

def model_status(epoch, accu, val_accu, loss, val_loss):
    test_path1 = 'Dataset/Test_Images/11740_sat.jpg'
    test_path2 = 'Dataset/Test_Images/112348_sat.jpg'
    test_path3 = 'Dataset/Test_Images/115172_sat.jpg'
    test_path4 = 'Dataset/Test_Images/173947_sat.jpg'

    test_img1  = np.asarray([imread(test_path1)])
    test_img2  = np.asarray([imread(test_path2)])
    test_img3  = np.asarray([imread(test_path3)])
    test_img4  = np.asarray([imread(test_path4)])

    f = plt.figure(figsize = (10, 14))
    gs = f.add_gridspec(4, 4)
    f.suptitle(f'Epoch: {epoch}', x=0.5, y=0.02)

    
    f.add_subplot(gs[0, 0])
    plt.imshow(imread(test_path1), cmap='gray')
    plt.title("Input Image")
    plt.axis('off')
    f.add_subplot(gs[0, 1])
    plt.imshow(model.predict(test_img1, verbose=1)[0][:,:,0], cmap='gray')
    plt.title("Predicted Image")
    plt.axis('off')

    f.add_subplot(gs[1, 0])
    plt.imshow(imread(test_path2), cmap='gray')
    plt.axis('off')
    f.add_subplot(gs[1, 1])
    plt.imshow(model.predict(test_img2, verbose=1)[0][:,:,0], cmap='gray')
    plt.axis('off')

    f.add_subplot(gs[2, 0])
    plt.imshow(imread(test_path3), cmap='gray')
    plt.axis('off')
    f.add_subplot(gs[2, 1])
    plt.imshow(model.predict(test_img3, verbose=1)[0][:,:,0], cmap='gray')
    plt.axis('off')

    f.add_subplot(gs[3, 0])
    plt.imshow(imread(test_path3), cmap='gray')
    plt.axis('off')
    f.add_subplot(gs[3, 1])
    plt.imshow(model.predict(test_img4, verbose=1)[0][:,:,0], cmap='gray')
    plt.axis('off')


    f.add_subplot(gs[0:2, 2:4])
    plt.plot(accu)
    plt.plot(val_accu)
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Training Accuracy', 'Validation accuracy'], loc='lower right')
    
    f.add_subplot(gs[2:4, 2:4])
    plt.plot(loss)
    plt.plot(val_loss)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Training loss', 'Validation loss'], loc='upper right')
    
    plt.savefig(f'epochs/{epoch}.png')
    plt.show()



class DisplayCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.accu = []
        self.val_accu = []
        self.loss = []
        self.val_loss = []
    def on_epoch_end(self, epoch, logs=None):
        self.accu.append(logs.get("accuracy"))
        self.val_accu.append(logs.get("val_accuracy"))
        self.loss.append(logs.get("loss"))
        self.val_loss.append(logs.get("val_loss"))
        clear_output(wait=True)
        model_status(epoch, self.accu, self.val_accu, self.loss, self.val_loss)

In [ ]:
EPOCHS = 100
LEARNING_RATE = 0.0001
BATCH_SIZE = 56
adam = tf.keras.optimizers.Adam(LEARNING_RATE)

model.compile(optimizer=adam, loss=soft_dice_loss, metrics=['accuracy'])

In [ ]:
history = model.fit(train_images,
                    train_masks,
                    validation_data = (test_images, test_masks),
                    epochs = EPOCHS,
                    batch_size = BATCH_SIZE,
                    callbacks = [checkpointer, DisplayCallback()])

In [ ]:
history_fig = plt.figure(figsize=(20,5))

accuracy = history_fig.add_subplot(1,2,1)
imgplot = plt.plot(history.history['accuracy'])
imgplot = plt.plot(history.history['val_accuracy'])
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training Accuracy', 'Validation accuracy'], loc='upper right')
accuracy.set_title("Epoch Accuracy")

loss = history_fig.add_subplot(1,2,2)
imgplot = plt.plot(history.history['loss'])
imgplot = plt.plot(history.history['val_loss'])
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training loss', 'Validation loss'], loc='upper right')
loss.set_title("Epoch Loss")

In [ ]:
import random
from skimage.io import imshow
from tensorflow.keras.models import load_model
from matplotlib import pyplot as plt


model = load_model("./Trained_Model/Road_Model.h5", custom_objects={'soft_dice_loss': soft_dice_loss, 'iou_coef': iou_coef})
predictions = model.predict(test_images, verbose=1)
thresh_val = 0.1
predicton_threshold = (predictions > thresh_val).astype(np.uint8)


ix = random.randint(0, len(predictions))
num_samples = 3

f = plt.figure(figsize = (12, 10))
for i in range(1, num_samples*4, 4):
    ix = random.randint(0, len(predictions))

    f.add_subplot(num_samples, 4, i)
    imshow(test_images[ix])
    plt.title("Image")
    plt.axis('off')

    f.add_subplot(num_samples, 4, i+1)
    imshow(np.squeeze(test_masks[ix][:,:,0]))
    plt.title("Ground Truth")
    plt.axis('off')

    f.add_subplot(num_samples, 4, i+2)
    imshow(np.squeeze(predictions[ix][:,:,0]))
    plt.title("Prediction")
    plt.axis('off')

plt.show()

test_path = 'Dataset/Images/100892_sat.jpg'
mask_path = 'Dataset/Masks/100892_mask.png'
test_img  = np.asarray([imread(test_path)])

f = plt.figure(figsize = (12, 10))
f.add_subplot(1,3,1)
imshow(imread(test_path))
plt.title("Input Image")
plt.axis('off')

f.add_subplot(1,3,2)
imshow(imread(mask_path))
plt.title("Ground Truth")
plt.axis('off')

f.add_subplot(1,3,3)
imshow(model.predict(test_img, verbose=1)[0][:,:,0])
plt.title("Prtedicted Image")
plt.axis('off')

plt.show()

In [ ]:
!rm -rf *.mp4 *.avi
import cv2
import os

video_name = 'video.avi'
epochs = os.listdir('epochs')
frame = cv2.imread('epochs/1.png')
height, width, layers = frame.shape
video = cv2.VideoWriter(video_name, 0, 15, (width,height))
for i in range(1, len(epochs)):
    video.write(cv2.imread(f'epochs/{i}.png'))
cv2.destroyAllWindows()
video.release()

!ffmpeg -i video.avi -c:v copy -c:a copy output.mp4
!ffmpeg -i output.mp4 -vcodec libx265 -crf 28 Epochs.mp4
!rm output.mp4
!printf "Video file ready : Epochs.mp4"